In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import random
from sklearn.model_selection import train_test_split
import tensorflow as tf 
import os

In [55]:
# 데이터셋 shape 다듬기
def make_dataset(data, label, window_size=20, shuffle=False):
    feature_list = []
    label_list = []

    for i in range(len(data) - window_size):
        feature_list.append(data.iloc[i:i+window_size])
        label_list.append(label.iloc[i+window_size])
#     temp = np.array([[i, j] for i, j in zip(feature_list, label_list)])
    
    return np.array(feature_list), np.array(label_list)

In [92]:
# 모델 정확도 측정하기
def pred_model(file_name, eps):
    # 파일 읽어오기
    df = pd.read_csv(file_name, index_col = 'Date')
#     print(df)

    # 데이터 정규화
    scaler = MinMaxScaler()  # 전체를 정규화
    # Senti_Score	Open	High	Low	Close	Volume	EachStock	KOSPI
    scale_cols = ['Open', 'High', 'Low', 'Close', 'Volume', 'EachStock', 'KOSPI', 'Senti_Score']
    df_scaled = scaler.fit_transform(df[scale_cols])
    df_scaled = pd.DataFrame(df_scaled)
    df_scaled.columns = scale_cols
#     print(df_scaled)

    scaler_close = MinMaxScaler()  # close만 정규화  ->  나중에 다시 되돌릴 때 필요
    df_scaled_close = scaler_close.fit_transform(df[['Close']])
    # scaler_close.inverse_transform(df_scaled_close)  # 정규화한 것을 되돌리기

    # 기본 사이즈 정하기
    TEST_SIZE = 100
    WINDOW_SIZE = 20
    
    test = df_scaled[-TEST_SIZE:]

    # 입력 데이터, 출력 데이터 나누기
    feature_cols = ['Senti_Score', 'Open', 'High', 'Low', 'Volume', 'EachStock', 'KOSPI']  # input
    label_cols = ['Close']  # output

    # 테스트 데이터 넣기
    train_feature = test[feature_cols]
    train_label = test[label_cols]
    X, y = make_dataset(train_feature, train_label, WINDOW_SIZE)

    # 모델 예측하기
    pred = load_model.predict(X)

    # 예측데이터 비교하기
#     plt.figure(figsize=(12, 9))
#     plt.plot(scaler_close.inverse_transform(y[:-1]), label = 'actual')
#     plt.plot(scaler_close.inverse_transform(pred[1:]), label = 'prediction')
#     plt.legend()
#     plt.show()
    
    cnt = 0
#     print(len(scaler_close.inverse_transform(y[:-1])))
    for i in range(len(pred[21:])):
        actual = scaler_close.inverse_transform(y[20:])
        prediction = scaler_close.inverse_transform(pred[21:])
        
        # 상승, 하락 맞추기
#         if (actual[i + 1] - actual[i] > 0) and (prediction[i + 1] - prediction[i] > 0):
#             cnt += 1
#         elif (actual[i + 1] - actual[i] < 0) and (prediction[i + 1] - prediction[i] < 0):
#             cnt += 1

        # 오차범위에 있는지
        if (actual[i] - prediction[i] if actual[i] >= prediction[i] else prediction[i] - actual[i]) <= actual[i] * eps:  # 오차범위 내라면
            cnt += 1
    
    return cnt, len(pred)

In [93]:
base_path = '../../dataset/데이터셋/기사'
news_list = [file for file in os.listdir(base_path) if 'csv' in file]
len(news_list)


# 기존에 학습시킨 모델 불러오기
load_model = tf.keras.models.load_model('../../dataset/Model/StockHelperModel.h5')

In [94]:
eps = [0.01, 0.03, 0.05] # 오차범위 1%, 3%, 5%

sum_result = 0.
total_cnt = 0 # 전체 날짜의 개수

result = []

for i in range(3):
    j = 0
    for data_set in news_list:
        diff, cnt = pred_model(os.path.join(base_path, data_set), eps[i])

        sum_result += diff
        total_cnt += cnt
        
        j+=1
        print(j, end=', ')
    print()
    
    result.append(int((sum_result / total_cnt) * 100))
    print(result)

1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 
[6]
1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 
[6, 14]
1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 8

In [70]:
result

[12, 24, 35]